In [29]:
source = "anva_meeus"
metadata_path = "Files/metadata/connection_anva_concern_prod_metadata.parquet"
log_to_console = True

In [30]:
from modules.logging_utils import configure_logging
import logging

log_file = configure_logging(run_name="fabric_metadata_generator", enable_console_logging=log_to_console)
logger = logging.getLogger(__name__)
logger.setLevel(logging.INFO)
logger.info("Logfile: %s", log_file)

2025-12-02 20:05:11,318 [INFO] - Logfile: /data/lakehouse/gh_b_avd/lh_gh_bronze/Files/notebook_outputs/logs/fabric_metadata_generator_20251202_185032.log


In [31]:
from modules.spark_session import get_or_create_spark_session

spark = get_or_create_spark_session(
    app_name="Test Spark Session",
    enable_hive=True
)

2025-12-02 20:05:11,343 [INFO] - ✓ Using existing Spark session
2025-12-02 20:05:11,344 [INFO] -   Spark version: 3.5.5
2025-12-02 20:05:11,344 [INFO] -   Application ID: app-20251202185052-0842
2025-12-02 20:05:11,345 [INFO] -   Application name: Fabric_Metadata_Generator


In [32]:
from modules.path_utils import resolve_files_path

resolved_metadata_path = resolve_files_path(metadata_path, spark)


metadata_df = spark.read.parquet(resolved_metadata_path)


2025-12-02 20:05:12,152 [INFO] - Detected cluster Files path: /data/lakehouse/gh_b_avd/lh_gh_bronze/Files


In [38]:
(
metadata_df 
    .select("server_name", "db_name", "obj_name", "column_name")
    .filter(metadata_df["obj_name"].isNotNull())
    .show(1000, truncate=False)
)

+-------------+----------------------+------------------------------------+-------------------------------------------------+
|server_name  |db_name               |obj_name                            |column_name                                      |
+-------------+----------------------+------------------------------------+-------------------------------------------------+
|vmdwhidpweu01|InsuranceData_ANVA_DWH|Dim_PolisVariabel_VrijeLabels       |Vrij_LabelGetal_4                                |
|vmdwhidpweu01|InsuranceData_ANVA_DWH|Dim_PolisVariabel_VrijeLabels       |Vrij_LabelGetal_5                                |
|vmdwhidpweu01|InsuranceData_ANVA_DWH|Dim_PolisVariabel_VrijeLabels       |Vrij_LabelGetal_1                                |
|vmdwhidpweu01|InsuranceData_ANVA_DWH|Dim_PolisVariabel_VrijeLabels       |Vrij_LabelGetal_2                                |
|vmdwhidpweu01|InsuranceData_ANVA_DWH|Dim_PolisVariabel_VrijeLabels       |Vrij_LabelGetal_3                          